In [19]:
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 31 17:41:07 2025

@author: mekie
"""


import os as os
import pandas as pd
import numpy as np
import datetime as dt
import dateutil as dtu
import date_functions as dtf




def derive_cash_flows(computation_date, maturity_date, freq, first_accrual_date, coupon_rate, spread, bond_type):
    """
    Computation date : date de calcul
    Maturity_dat : maturité de l'obigation
    freq: fréquence annuelle de l'obligation
    first_accrual_date : date de début et date de naissance des intérets
    coupon_rate: taux de coupon en pourcentage du nominal
    spread: pour des bonds flotteurs spread à rajouter sur le taux de coupon flotteur
    bond_type : type de d'obligation ici nous avons zero_coupon, fixe, flotteurs, indexé sur l'inflation
    """
    if freq!=0: #cas NON zero coupon
        nb_init_cf = max(1, int((maturity_date - first_accrual_date).days/365) * freq) #nombre de cashflow
        cashFlows = pd.DataFrame({'cf_id':np.int64(np.arange(nb_init_cf))})
        #les différentes dates de cas_flow obtenu en prenant la date de maturité et en rentrant en fonction de la fréquence (si la fréquence c'est 1 on rentre de 1 ans, si 2 on rentre de 6 mois)
        cashFlows['cf_dates'] = cashFlows.apply(lambda x : dtf.add_dmy(maturity_date, 0, -12*x.cf_id/freq, 0)  , axis=1)
        #période de temps en jours entre les dates de cashflow et et la date de naissance des intérets de l'obligation
        cashFlows['cf_accrual'] = cashFlows.apply(lambda x : (x['cf_dates'] - first_accrual_date).days, axis=1)
        #date dans lesquelles nous auront des futures cashflow (après la computation_date)
        cashFlows['upcoming_cf'] = 0
        cashFlows.loc[cashFlows.cf_dates>=computation_date, 'upcoming_cf'] = 1
        #durré en jours entre la date de pricing et les différentes dates de cashflows
        cashFlows['Ti'] = cashFlows.apply(lambda x : (x['cf_dates'] - computation_date).days, axis=1)
        #le nomnal (vaut 100 par défaut et est ajouté à la maturité)
        cashFlows['add_cf'] = 0; cashFlows.loc[0,'add_cf'] = 100
        #taux de coupon
        cashFlows['coupon_rate'] =coupon_rate
        #spread de coupon
        cashFlows['spread'] =spread
        #type d'obligation
        cashFlows['bond_type'] =bond_type
        #ne considérer que les periodes après date de naissance d'intéret
        cashFlows = cashFlows[cashFlows.cf_accrual>0]
    else:
        cashFlows = pd.DataFrame.from_dict({'cf_id':0}, orient='index')
        cashFlows['cf_dates'] = maturity_date
        cashFlows['cf_accrual'] = cashFlows.apply(lambda x : (x['cf_dates'] - first_accrual_date).days, axis=1)
        cashFlows['upcoming_cf'] = 0
        cashFlows.loc[cashFlows.cf_dates>=computation_date, 'upcoming_cf'] = 1
        cashFlows['Ti'] = cashFlows.apply(lambda x : (x['cf_dates'] - computation_date).days, axis=1)
        cashFlows['add_cf'] = 100
        cashFlows['coupon_rate'] =coupon_rate
        cashFlows['spread'] =spread
        cashFlows['bond_type'] =bond_type
        cashFlows = cashFlows[cashFlows.cf_accrual>0]  
    return cashFlows


#fonction permettant de genrer des bonds
def generate_n_bonds(nb_bonds = 10, bnd_ctry='FR', computation_date_dt = dt.date.today()):
    bond_df = pd.DataFrame.from_dict({'bond_id' : range(nb_bonds)})
    isin_unique_bit_list = np.random.choice(np.arange(int(1E5-1),int(1E8-1)),nb_bonds) # ,replace=False)
    bond_df['ISIN'] = [ "".join(np.r_[[bnd_ctry],np.repeat("0",12-2-len(str(ISIN_END))), [str(ISIN_END)]]) for ISIN_END in isin_unique_bit_list ]
    bond_df['accrual_per'] = np.random.uniform(0,30,nb_bonds)
    bond_df['TTM'] = np.random.uniform(0,30,nb_bonds)
    bond_df['initial_TTM'] = bond_df.apply(lambda x : int(min(round((x['accrual_per']+x['TTM'])), 60)), axis=1)
    bond_df['TTM'] = bond_df.apply(lambda x : (x['initial_TTM']- x['accrual_per']), axis=1)
    bond_df['first_accrual_date'] = bond_df.apply(lambda x : dtf.adjust_date(dtf.add_dmy(computation_date_dt, -int((x.accrual_per-int(x.accrual_per)) * 365), 0, -int(x.accrual_per) )), axis=1)
    bond_df['first_coupon_date'] = pd.to_datetime(np.nan)
    bond_df['maturity_date'] = bond_df.apply(lambda x : dtf.adjust_date(dtf.add_dmy(computation_date_dt, int((x.TTM-int(x.TTM)) * 365), 0, int(x.TTM) )), axis=1)
    bond_df['TTM'] = bond_df.apply(lambda x : (x['maturity_date']- computation_date_dt).days/365, axis=1)
    bond_df['coupon_rate'] = np.random.uniform(0,5,nb_bonds)
    bond_df['prices'] = np.random.uniform(98,120,nb_bonds)
    bond_df['freq'] = np.random.choice(np.arange(1,3),nb_bonds)
    bond_type_universe = ['zero', 'fix', 'infla', 'float']
    bond_df['bond_type'] = np.random.choice(bond_type_universe,nb_bonds)
    bond_df['freq'] = bond_df.apply(lambda x : [0 if x.bond_type=='zero' else 2 if x.bond_type=='float' else 1][0], axis=1)
    bond_df['spread'] = bond_df.apply(lambda x : [x.coupon_rate/200 if x.bond_type=='float' else np.nan][0], axis=1)
    bond_df['convention'] = bond_df.apply(lambda x : ["ACT/360" if x.bond_type=='float' else "ACT/ACT"][0], axis=1)
    bond_df['coupon_rate'] = bond_df.apply(lambda x : [0 if x.bond_type=='zero' else np.nan if x.bond_type=='float' else x.coupon_rate][0], axis=1)
    # bond_df['first_coupon_date'] = bond_df.apply(lambda x : [np.nan if x.freq==0 else derive_cash_flows(computation_date_dt, x.maturity_date, x.freq, x.first_accrual_date, x.coupon_rate, x.spread, x.bond_type).query("upcoming_cf==1").cf_dates.iloc[-1]][0], axis=1)
    bond_df['first_coupon_date'] = bond_df.apply(lambda x : [np.nan if x.freq==0 else derive_cash_flows(computation_date_dt, x.maturity_date, x.freq, x.first_accrual_date, x.coupon_rate, x.spread, x.bond_type).cf_dates.iloc[-1]][0], axis=1)
    return bond_df
